In [1]:
%run 1.common.ipynb

/root/anaconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def f(data):
    #合并x,y
    data = [torch.cat(i) for i in data]

    #求最大长度
    lens = max([len(i) for i in data])

    #做个白板
    input_ids = torch.full((len(data), lens), tokenizer.encoder['PAD'])

    #往白板里黏贴数据
    for i, d in enumerate(data):
        input_ids[i, :len(d)] = d

    attention_mask = (input_ids != tokenizer.encoder['PAD']).long()

    return input_ids, attention_mask


loader = get_loader(f)

next(iter(loader))

(tensor([[ 1,  5,  3,  ...,  0,  0,  0],
         [ 1, 11,  6,  ...,  0,  0,  0],
         [ 1, 16,  8,  ...,  0,  0,  0],
         ...,
         [ 1,  5,  3,  ...,  0,  0,  0],
         [ 1, 16,  6,  ...,  0,  0,  0],
         [ 1, 11, 11,  ...,  0,  0,  0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]))

In [3]:
@torch.no_grad()
def test(N=5, device='cpu'):
    for _ in range(N):
        x, y = tokenizer.get_data()
        print(tokenizer.decode(x + y))

        x = torch.LongTensor(x).unsqueeze(0).to(device)
        out = generate_model.generate(x, max_length=40)[0]
        print(tokenizer.decode(out))
        
        #尝试执行计算
        try:
            out = out[1:-1]
            idx_eq = (out == tokenizer.encoder['=']).nonzero().item() + 1
            out = tokenizer.decode(out[idx_eq:])
            print(eval(out))
        except:
            pass

        print('---------------')


test()

SOS-76.96=-46.97-29.99EOS
SOS-76.96=07-c7.1ccccc1c1c1c1c1c1c1c1c1111
---------------
SOS0.92=64.12**-0.02EOS
SOS0.92=5c15c1c1c1111111111111111111111111
---------------
SOS0.26=19.84/75.84EOS
SOS0.26=5c11111111111111111111111111111111
---------------
SOS-1.48=-90.08/61.04EOS
SOS-1.48=.1c1c1c1c111111111111111111111111
---------------
SOS-2569.27=-14.58**2.93EOS
SOS-2569.27=0707070707.=05c15c15c1c1c1c1c1
---------------


In [4]:
def train():
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss(
        ignore_index=tokenizer.encoder['PAD'])

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.train()
    model.to(device)

    for i in range(10000):
        input_ids, attention_mask = next(iter(loader))
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        out = model(input_ids=input_ids, attention_mask=attention_mask)

        input_ids = input_ids[:, 1:].flatten()
        out = out[:, :-1].flatten(end_dim=1)
        loss = criterion(out, input_ids)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 500 == 0:
            print(i)
            test(2, device)

    model.to('cpu')
    torch.save(model, 'sft.model')


train()

0
SOS163.00=83.00--80.00EOS
SOS163.00=01111111111111111111111111111111
---------------
SOS-0.00=-98.92**-2.56EOS
SOS-0.00=555555555522222EOS
555555555522222
---------------


/root/anaconda3/envs/llama/lib/python3.9/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


500
SOS-107.35=-61.78-45.57EOS
SOS-107.35=-14.77--15.76EOS
0.9900000000000002
---------------
SOS1.15=-95.90/-83.34EOS
SOS1.15=-10.00/-10.0EOS
1.0
---------------
1000
SOS19.96=-5.86+25.82EOS
SOS19.96=-7.59--10.07EOS
2.4800000000000004
---------------
SOS0.15=-10.04/-69.23EOS
SOS0.15=-2.07/-25.78EOS
0.08029480217222652
---------------
1500
SOS1024.33=-18.74*-54.66EOS
SOS1024.33=-73.90*-20.00EOS
1478.0
---------------
SOS-0.00=-54.98**-1.77EOS
SOS-0.00=-44.99**-1.99EOS
-0.0005132148018997137
---------------
2000
SOS-1.66=-49.99--48.33EOS
SOS-1.66=-13.13/11.02EOS
-1.1914700544464611
---------------
SOS-143.60=-53.76-89.84EOS
SOS-143.60=-93.11+-43.14EOS
-136.25
---------------
2500
SOS-7.91=90.23+-98.14EOS
SOS-7.91=-26.04--1.00EOS
-25.04
---------------
SOS-94.04=-66.21-27.83EOS
SOS-94.04=-6.18+-40.14EOS
-46.32
---------------
3000
SOS0.00=49.28**-3.44EOS
SOS0.00=99.90**-2.97EOS
1.1515704178655367e-06
---------------
SOS-266.23=-58.92**1.37EOS
SOS-266.23=-7.52**2.26EOS
-95.55475861429069
